# Downloading Libraries

In [1]:
! pip install nltk
! pip install vaderSentiment
! pip install pytrends
! pip install textblob
! pip install wordcloud
! pip install gensim
! pip install seaborn
! pip install TextBlob
! pip install joblib
! pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 1.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 6.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 11.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.1/330.1 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 392.9/392.9 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 10.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.0/169.0 kB 22.9 MB/s eta 0:00:00
  Created wheel for pyyaml: filename=PyYAML-6.0.1-cp38-cp38-macosx_11_0_arm64.whl size=45365 sha256=d94a14789ea5a39635a0e4c4c041080098f5f005dcb5b03b59e0f25523507a68
  Stored in directory: /Users/taimourabdulkarim/Library/Caches/pip/wheels/77/54/77/68b3079bd1d88cb070513c3935d9f

# Importing Libraries

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, TFBertForSequenceClassification
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

/Users/taimourabdulkarim/miniforge3/envs/mlp/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Importing Dataset

In [28]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [29]:
df = pd.read_csv('/content/drive/MyDrive/Datasets/Cleaned Data/reviews.csv')
df.head()

,listing_id,reviewer_name,comments,cleaned_comments,polarity,sentiment
0,4326511,Laura,Das Zimmer und das Bad waren sauber und komfor...,da zimmer und da bad waren sauber und komforta...,-0.7,negative
1,603032069621870277,Fatima,Bad service,bad servic,-0.7,negative
2,8629818,Andrew,"In Paul's absence, Lucy and Jack were amazing ...",paul absenc luci jack amaz host brthe secret g...,-0.4,negative
3,25012636,Kate,I used Nicholas’ room as a base whilst I was i...,use nicholas’ room base whilst properti busi f...,-0.8,negative
4,12725143,Yvon,You can’t go wrong with the townhouse. It has ...,can’t go wrong townhous amen need place spotle...,-0.5,negative


# Date Preprocessing

In [30]:
df.isnull().sum()

listing_id            0
reviewer_name        63
comments             63
cleaned_comments    209
polarity            163
sentiment           163
dtype: int64

In [31]:
# Dropping the null values
df.dropna(inplace=True)
df.isnull().sum()

listing_id          0
reviewer_name       0
comments            0
cleaned_comments    0
polarity            0
sentiment           0
dtype: int64

## **<center><u>Assessment: Experimentations (using Deep Learning)</u></center>**


In [32]:
# Initialize tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [34]:
# Tokenize and encode sequences in the dataset
tokens = tokenizer.batch_encode_plus(
    df['cleaned_comments'].tolist(),
    max_length=128,  # You can adjust this depending on your text length
    padding='max_length',
    truncation=True,
    add_special_tokens=True,
    return_attention_mask=True,
    return_tensors='tf'
  )

In [35]:
# Prepare input data
input_ids = tokens['input_ids']
attention_mask = tokens['attention_mask']

In [36]:
# Prepare labels
df['sentiment'] = df['sentiment'].astype('category')
labels = df['sentiment'].cat.codes

In [37]:
input_ids_np = input_ids.numpy()
attention_mask_np = attention_mask.numpy()

labels_np = labels.to_numpy()

In [38]:
train_input_ids, test_input_ids, train_attention_mask, test_attention_mask, train_labels, test_labels = train_test_split(
    input_ids_np,
    attention_mask_np,
    labels_np,  # assuming labels_np is already a numpy array
    test_size=0.2,
    random_state=42
)

In [39]:
train_input_ids = tf.convert_to_tensor(train_input_ids)
train_attention_mask = tf.convert_to_tensor(train_attention_mask)
train_labels = tf.convert_to_tensor(train_labels)

test_input_ids = tf.convert_to_tensor(test_input_ids)
test_attention_mask = tf.convert_to_tensor(test_attention_mask)
test_labels = tf.convert_to_tensor(test_labels)

In [40]:
train_labels = tf.cast(train_labels, dtype=tf.int32)

In [41]:
model = TFBertForSequenceClassification.from_pretrained(
    'bert-base-uncased', num_labels=len(df['sentiment'].cat.categories))

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [42]:
optimizer = Adam(learning_rate=2e-5, epsilon=1e-08)
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [43]:
history = model.fit(
    {'input_ids': train_input_ids, 'attention_mask': train_attention_mask},
    train_labels,
    batch_size=32,  # Adjust based on your GPU memory
    epochs=10,  # BERT typically requires only a few epochs
    validation_split=0.1  # Use a part of the training set for validation
)

Epoch 1/10
261/261 [==============================] - 252s 803ms/step - loss: 1.1124 - accuracy: 0.3397 - val_loss: 1.0986 - val_accuracy: 0.3136
Epoch 2/10
261/261 [==============================] - 209s 800ms/step - loss: 1.0986 - accuracy: 0.3413 - val_loss: 1.0986 - val_accuracy: 0.3136
Epoch 3/10
261/261 [==============================] - 209s 800ms/step - loss: 1.0986 - accuracy: 0.3413 - val_loss: 1.0986 - val_accuracy: 0.3136
Epoch 4/10
261/261 [==============================] - 206s 790ms/step - loss: 1.0986 - accuracy: 0.3413 - val_loss: 1.0986 - val_accuracy: 0.3136
Epoch 5/10
261/261 [==============================] - 209s 800ms/step - loss: 1.0986 - accuracy: 0.3413 - val_loss: 1.0986 - val_accuracy: 0.3136
Epoch 6/10
261/261 [==============================] - 206s 789ms/step - loss: 1.0986 - accuracy: 0.3413 - val_loss: 1.0986 - val_accuracy: 0.3136
Epoch 7/10
261/261 [==============================] - 206s 790ms/step - loss: 1.0986 - accuracy: 0.3413 - val_loss: 1.0986 -

In [44]:
result = model.evaluate(
    x={'input_ids': test_input_ids, 'attention_mask': test_attention_mask},
    y=test_labels
)
print(f"Test loss: {result[0]}, Test accuracy: {result[1]}")

73/73 [==============================] - 22s 264ms/step - loss: 1.0986 - accuracy: 0.3272
Test loss: 1.0986123085021973, Test accuracy: 0.3271551728248596
